### A coding scheme for Simon game sequences

#### **Author:** Sol Markman (smarkman@mit.edu)

Assumption: This scheme is hierarchical, so that different chunk types take priority over others. For example, repeats of a single color are chunked first, then repeats of multiple colors, then nested repeats, and then alternations are chunked with the remaining items, etc.
- RRRGRGRG is compressed to [R]^3 [GR]^2 G, size=6
- RRRGRG is compressed to [R]^3 [GRG], size=4

#### Types of chunks/compression and their sizes:

#### 1) Repeats
Size = (size of subsequence) + 1. This assumes that size does not increase with the number of repeats, and 
repeats are only beneficial for size>2.

a) Single color repeats
- RRRR is compressed to [R]^4, size=2

b) Multicolor repeats
- RGRGRG is compressed to [RG]^3, size=3
    
c) Nested repeats
- RRRGGGRRRGGG is compressed to [ [R]^3 [G]^3 ]^2, size=5

#### 2) Alternations
Not yet implemented

#### 3) Cycles
Not yet implemented

#### 4) Exposure compression (within a sequence)
Not yet implemented


In [2]:
# Repeat chunking for hypotheses 1 (single repeats only) and 3 (+ multicolor and nested repeats).
def find_repeats(seq, length): 
    """
    Finds repeats of patterns of a fixed length.
    """
    i = 0
    result = []
    while i < len(seq):
        if i + 2 * length <= len(seq) and seq[i:i+length] == seq[i+length:i+2*length]:
            pattern = seq[i:i+length]
            count = 2
            j = i + 2 * length
            while j + length <= len(seq) and seq[j:j+length] == pattern:
                count += 1
                j += length
            result.append(f'[{pattern}]{count}')
            i = j

        else: # move onto the next character
            result.append(seq[i])
            i += 1

    return ''.join(result)

def repeat_chunking(seq, single_only=False): 
    '''
    Finds repeating patterns within a sequence recursively.
    If single_only, only simple single-character repeats.
    Otherwise, includes multicolor and nested repeats.
    Note: Shorter pattern lengths are chunked first. Once a character is chunked, that chunk cannot be broken.
        E.g. GBGBBB = GBG[B]3, not [GB]2[B]2 because the B's at the end are chunked first.
    '''
    length = 1 # Initialize pattern length
    max_len = 1

    # recursively look for repeating patterns of increasing length
    new_seq = seq
    while length <= max_len:
        new_seq = find_repeats(new_seq, length)
        max_len = 1 if single_only else len(new_seq)//2
        length += 1 # increase length and keep going

    code = ''.join(new_seq)
    code_len = len(code.replace('[', '').replace(']', ''))
    chunkability = 1 - code_len/len(seq)

    return code_len, chunkability, code

In [5]:
# coding scheme 1: chunking simple (single character) repeats only.
def hyp1(seq):
    return repeat_chunking(seq, single_only=True)

# Testing hyp1 code
test_sequences = ['BBBRRRYYYGGG', 'RRRZGZRRR', 'BBBBGRY', 'RRRYGYGYG', 'GBGBB', 
                    'BBBBBBBrrrBBBBBB', 'GGGGGGGG', 'ABCD', 'BGGGR', 'abbabbabba']
answers = [8, 7, 5, 8, 5, 6, 2, 4, 4, 10]

def test_coding_scheme(scheme, sequences, answers):
    for i in range(len(sequences)):
        s_idx = i
        sequence = sequences[s_idx]
        print(sequence, answers[s_idx])
        print(scheme(sequence))
        if scheme(sequence)[0] != answers[s_idx]:
            print('^ERROR')
    

test_coding_scheme(hyp1, test_sequences, answers)

BBBRRRYYYGGG 8
(8, 0.33333333333333337, '[B]3[R]3[Y]3[G]3')
RRRZGZRRR 7
(7, 0.2222222222222222, '[R]3ZGZ[R]3')
BBBBGRY 5
(5, 0.2857142857142857, '[B]4GRY')
RRRYGYGYG 8
(8, 0.11111111111111116, '[R]3YGYGYG')
GBGBB 5
(5, 0.0, 'GBG[B]2')
BBBBBBBrrrBBBBBB 6
(6, 0.625, '[B]7[r]3[B]6')
GGGGGGGG 2
(2, 0.75, '[G]8')
ABCD 4
(4, 0.0, 'ABCD')
BGGGR 4
(4, 0.19999999999999996, 'B[G]3R')
abbabbabba 10
(10, 0.0, 'a[b]2a[b]2a[b]2a')


In [6]:
# coding scheme 3: including complex repeats (multi-item and nesting).
def hyp3(seq):
    return repeat_chunking(seq, single_only=False)

# Testing hyp3 code
test_sequences = ['BBBRRRYYYGGG', 'RRRZGZRRR', 'BBBBGRY', 'RRRYGYGYG', 'GBGBB', 
                'BBBBBBBrrrBBBBBB', 'GRGRGRZDZDZDZTTTTTTTGGG', 'ababcdcdababcdcd',
                'abbabbabba', 'abcabcabc', 'abbccccabbccccabbcccc', 'bbbbaaaabbbbaaaa',
                'bbbaaabbbaaa', 'bbaabbaa', 'baba', 'bbaabbaab']
answers = [8, 7, 5, 5, 5, 6, 11, 7, 5, 4, 6, 5, 5, 5, 3, 6]

test_coding_scheme(hyp3, test_sequences, answers)

BBBRRRYYYGGG 8
(8, 0.33333333333333337, '[B]3[R]3[Y]3[G]3')
RRRZGZRRR 7
(7, 0.2222222222222222, '[R]3ZGZ[R]3')
BBBBGRY 5
(5, 0.2857142857142857, '[B]4GRY')
RRRYGYGYG 5
(5, 0.4444444444444444, '[R]3[YG]3')
GBGBB 5
(5, 0.0, 'GBG[B]2')
BBBBBBBrrrBBBBBB 6
(6, 0.625, '[B]7[r]3[B]6')
GRGRGRZDZDZDZTTTTTTTGGG 11
(11, 0.5217391304347826, '[GR]3[ZD]3Z[T]7[G]3')
ababcdcdababcdcd 7
(7, 0.5625, '[[ab]2[cd]2]2')
abbabbabba 5
(5, 0.5, '[a[b]2]3a')
abcabcabc 4
(4, 0.5555555555555556, '[abc]3')
abbccccabbccccabbcccc 6
(6, 0.7142857142857143, '[a[b]2[c]4]3')
bbbbaaaabbbbaaaa 5
(5, 0.6875, '[[b]4[a]4]2')
bbbaaabbbaaa 5
(5, 0.5833333333333333, '[[b]3[a]3]2')
bbaabbaa 5
(5, 0.375, '[[b]2[a]2]2')
baba 3
(3, 0.25, '[ba]2')
bbaabbaab 6
(6, 0.33333333333333337, '[[b]2[a]2]2b')
